# 卷积层

到目前为止，我们已经通过引入展平层，使神经网络具备了处理多维数据（如图像）的能力；并通过引入多样化的激活函数和损失函数，使其更适用于分类任务。

但是，我们目前的网络模型对于处理图像还存在明显的缺陷。

-----------------

当前我们一直使用的是最经典的**全连接神经网络**（Fully Connected Neural Network）。其中每一层的每个神经元都与前一层的所有神经元相连。这意味着同样的数值在不同的位置上代表着不同的含义。

比如：在小明的冰激凌销量预测模型里，20 这个数值，出现在第一个位置代表气温为 20°C；出现在第二个位置代表湿度为 20%。这种对位置高度敏感的建模方式，在结构化数值数据中是合理的。

但在图像处理中，情况却大不相同。图像中的对象往往具有**平移不变性**（translation invariance）：一个小球无论出现在图像的左上角还是右下角，本质上仍是同一个物体。使用全连接神经网络，会忽略这种局部结构的共性。

另外，对于多维数据进行全连接，会造成参数数量爆炸而难以训练。

-----------------

为了解决这些问题，研究者引入了**卷积层**（Convolutional Layer），一种专为处理具有网格结构的数据（如图像、音频、视频）而设计的神经网络组件。一般认为，日本学者福岛邦彦（Kunihiko Fukushima）在1980年提出的**新识别机**（Neocognitron）是最早的卷积结构。随后，杨立昆（Yann LeCun）等人在1989年将反向传播算法与卷积结构结合，奠定了现代卷积神经网络的基础。

-----------------

卷积层的核心思想是：用一个小型的权重数组，也称**卷积核**（kernel）或权重模板，在输入数据上滑动，提取局部特征。

* 假设输入数据是一张 4 × 4 的二维图像：

```{figure} images/kernel-1.png
:align: center
:width: 190px
```

* 卷积层可以定义一个尺寸较小的卷积核（例如：2 × 2）：

```{figure} images/kernel-2.png
:align: center
:width: 105px
```

* 该卷积核在输入数据上逐行逐列滑动：

```{figure} images/kernel-3.png
:align: center
:width: 640px
```

* 每次与对应区域进行加权求和，得到该区域的卷积值：

```{figure} images/kernel-4.png
:align: center
:width: 180px
```

* 最终获得一张由卷积值组成的输出数据：

```{figure} images/kernel-5.png
:align: center
:width: 120px
```

卷积层的关键特性包括：

* **局部连接**（Local Connectivity）：每个输出卷积值仅与输入的一个局部区域连接；
* **权值共享**（Weight Sharing）：同一个卷积核在整个输入上共享参数。

这是实现平移不变性的关键，同时也大幅减少模型参数量。

通过堆叠多个卷积层，网络能够从原始像素逐步构建出层次化的视觉表示，从而高效地完成图像分类、目标检测、语义分割等任务。

In [1]:
from abc import abstractmethod, ABC
import numpy as np

np.random.seed(99)

## 基础架构

### 张量

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    @property
    def size(self):
        return np.prod(self.data.shape[1:])

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础数据集

In [3]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size

        self.test_labels = self.test_features = None
        self.train_labels = self.train_features = None

        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size, Tensor(self.labels).size

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

    def estimate(self, predictions):
        pass

### 基础层

In [4]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, x: Tensor):
        return self.forward(x)

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    @property
    def parameters(self):
        return []

    def __repr__(self):
        return ''

### 基础损失函数

In [5]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

In [6]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def reset(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

### 基础模型

In [7]:
class Model(ABC):

    def __init__(self, layer, loss_fn, optimizer):
        self.layer = layer
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## 数据

### MNIST 数据集

In [8]:
class MNISTDataset(Dataset):

    def __init__(self, filename, batch_size=1):
        self.filename = filename
        super().__init__(batch_size)

    def load(self):
        with (np.load(self.filename, allow_pickle=True) as f):
            self.train_features, self.train_labels = self.normalize(f['x_train'], f['y_train'])
            self.test_features, self.test_labels = self.normalize(f['x_test'], f['y_test'])

    @staticmethod
    def normalize(x, y):
        inputs = x / 255
        inputs = np.expand_dims(inputs, axis=1)
        targets = np.zeros((len(y), 10))
        targets[range(len(y)), y] = 1
        return inputs, targets

    def estimate(self, predictions):
        count = (predictions.data.argmax(axis=1) == self.labels.argmax(axis=1)).sum()
        total = len(self.labels)
        return count / total

## 模型

### 线性层

In [9]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.weight = Tensor(np.random.rand(out_size, in_size) / in_size)
        self.bias = Tensor(np.random.rand(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data
            self.bias.grad += np.sum(p.grad, axis=0)
            x.grad += p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    @property
    def parameters(self):
        return [self.weight, self.bias]

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 顺序层

In [10]:
class Sequential(Layer):

    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def train(self):
        for l in self.layers:
            l.train()

    def eval(self):
        for l in self.layers:
            l.eval()

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    @property
    def parameters(self):
        return [p for l in self.layers for p in l.parameters]

    def __repr__(self):
        return '\n'.join(str(l) for l in self.layers if str(l))

### 卷积层

**前向传播**：在一个典型的卷积层前向传播中，假设输入值为 $X$，卷积核权重为 $W$，偏置为 $b$，输出值 $Y$ 的计算过程如下：

$$
Y_{i,j} = \sum_{m=0}^{k-1} \sum_{n=0}^{k-1} X_{i+m, j+n} \cdot W_{m,n} + b
$$

其中 $k$ 为卷积核的空间尺寸（假设为 $k*k$）。卷积层输出图像的尺寸遵循公式：

$$
\text{output size} = \text{input size} - k + 1
$$

实际图像数据通常还包含颜色通道（比如：RGB 三通道），因此输入数据实际上是三维结构（高度、宽度、通道数）。相应地，卷积核也扩展为三维结构（和输入数据相同的通道数），而每个输出值是对所有输入通道卷积结果求和后得到的一个标量。

**梯度计算**：卷积层的梯度计算本质上也是一个反卷积操作。具体而言，就是对后层梯度依次进行梯度计算，卷积核的权重和偏置由每个卷积步骤的结果累积而成。

**父节点列表**（parents）：输入值。

**参数列表**（parameters）：卷积核权重和偏置。

---

以上是一个卷积核的操作过程。

每个卷积层可以有多个卷积核，每个卷积核有自己的权重和偏置。每个卷积核输出一张卷积图像，成为最终输出结果的一个通道。卷积层有几个卷积核，输出结果就有几个通道。

比如：MNIST 图像只有一个颜色通道，像素是 1 × 28 × 28。如果我们卷积层有 16 个 3 × 3 的卷积核，每个卷积核生成的卷积图像的像素是 1 × 26 × 26。所有卷积核生成的卷积图像合并在一起，最终的输出结果就是一个 16 通道的卷积图像，像素是 16 × 26 × 26。

-----------------

实践中，卷积层的计算通常会采用**图转列**（im2col）技术来充分利用 NumPy 的矢量并行运送能力。

简单地讲，就是对卷积核依次经过的输入区域进行一次展平操作，转换成一维向量；

然后把这些向量拼接到一个大的二维矩阵；

最后把卷积核也展平成一个一维向量，用它对二维矩阵进行矩阵乘法。一次性得到所有卷积步骤的结果。这种方法是以空间换时间，能显著提升计算效率。

In [11]:
class Convolution(Layer):

    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels    # 卷积核数
        self.kernel_size = kernel_size      # 卷积核大小
        in_size = in_channels * kernel_size * kernel_size   # 卷积核特征维度
        self.weight = Tensor(np.random.rand(out_channels, in_channels, kernel_size, kernel_size) * np.sqrt(2 / in_size))
        self.bias = Tensor(np.zeros(out_channels))

    def forward(self, x: Tensor):
        batch, in_ch, in_h, in_w = x.data.shape
        out_h = in_h - self.kernel_size + 1
        out_w = in_w - self.kernel_size + 1
        x_padded = x.data

        # 图转列
        patches = []
        for i in range(out_h):
            for j in range(out_w):
                patch = x_padded[:, :, i:i + self.kernel_size, j:j + self.kernel_size]
                patches.append(patch)
        patches = np.array(patches).transpose(1, 0, 2, 3, 4)
        patches_reshaped = patches.reshape(batch, out_h * out_w, -1)
        weight_reshaped = self.weight.data.reshape(self.out_channels, -1)
        output = patches_reshaped @ weight_reshaped.T + self.bias.data
        output = output.reshape(batch, out_h, out_w, self.out_channels)
        output = output.transpose(0, 3, 1, 2)
        p = Tensor(output)

        def gradient_fn():
            grad_output = p.grad.transpose(0, 2, 3, 1).reshape(batch, out_h * out_w, self.out_channels)
            weight_grad = grad_output.reshape(-1, self.out_channels).T @ patches_reshaped.reshape(-1, patches_reshaped.shape[-1])

            self.weight.grad += weight_grad.reshape(self.weight.data.shape) / batch
            self.bias.grad += np.sum(grad_output, axis=(0, 1)) / batch

            input_grad = grad_output @ weight_reshaped
            input_grad = input_grad.reshape(batch, out_h, out_w, in_ch, self.kernel_size, self.kernel_size)

            grad_input = np.zeros_like(x_padded)
            idx = 0
            for i in range(out_h):
                for j in range(out_w):
                    grad_input[:, :, i:i + self.kernel_size, j:j + self.kernel_size] += input_grad[:, i, j, :, :, :]
                    idx += 1
            x.grad += grad_input / batch

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    @property
    def parameters(self):
        return [self.weight, self.bias]

    def __repr__(self):
        return f'Convolution[weight{self.weight.data.shape}; bias{self.bias.data.shape}; kernels={self.out_channels}, kernel size={self.kernel_size}]'

### 展平层

In [12]:
class Flatten(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.array(x.data.reshape(x.data.shape[0], -1)))

        def gradient_fn():
            x.grad += p.grad.reshape(x.data.shape)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'Flatten[]'

### 丢弃层

In [13]:
class Dropout(Layer):

    def __init__(self, dropout_rate=0.2):
        super().__init__()
        self.dropout_rate = dropout_rate

    def forward(self, x: Tensor):
        if not self.training:
            return x

        mask = np.random.random(x.data.shape) > self.dropout_rate
        p = Tensor(x.data * mask)

        def gradient_fn():
            x.grad += p.grad * mask

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'Dropout[rate={self.dropout_rate}]'

### ReLU 激活函数

In [14]:
class ReLU(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.maximum(0, x.data))

        def gradient_fn():
            x.grad += p.grad * (p.data > 0)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'ReLU[]'

### Tanh 激活函数

In [15]:
class Tanh(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.tanh(x.data))

        def gradient_fn():
            x.grad += p.grad * (1 - p.data ** 2)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'Tanh[]'

### Sigmoid 激活函数

In [16]:
class Sigmoid(Layer):

    def __init__(self, clip_range=(-100, 100)):
        super().__init__()
        self.clip_range = clip_range

    def forward(self, x: Tensor):
        z = np.clip(x.data, self.clip_range[0], self.clip_range[1])
        p = Tensor(1 / (1 + np.exp(-z)))

        def gradient_fn():
            x.grad += p.grad * p.data * (1 - p.data)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'Sigmoid[]'

### Softmax 激活函数

In [17]:
class Softmax(Layer):

    def __init__(self, axis=-1):
        super().__init__()
        self.axis = axis

    def forward(self, x: Tensor):
        exp = np.exp(x.data - np.max(x.data, axis=self.axis, keepdims=True))
        p = Tensor(exp / np.sum(exp, axis=self.axis, keepdims=True))

        def gradient_fn():
            grad = np.sum(p.data * p.grad, axis=self.axis, keepdims=True)
            x.grad += p.data * (p.grad - grad)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'Softmax[]'

### 损失函数（均方误差）

In [18]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data) / len(y.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### 损失函数（交叉熵）

In [19]:
class CELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        exp = np.exp(p.data - np.max(p.data, axis=-1, keepdims=True))
        softmax = exp / np.sum(exp, axis=-1, keepdims=True)

        log = np.log(np.clip(softmax, 1e-10, 1))
        ce = Tensor(0 - np.sum(y.data * log) / len(y.data))

        def gradient_fn():
            p.grad += (softmax - y.data) / len(y.data)

        ce.gradient_fn = gradient_fn
        ce.parents = {p}
        return ce

### 损失函数（二元交叉熵）

In [20]:
class BCELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        clipped = np.clip(p.data, 1e-7, 1 - 1e-7)
        bce = Tensor(-np.mean(y.data * np.log(clipped) + (1 - y.data) * np.log(1 - clipped)))

        def gradient_fn():
            p.grad += (clipped - y.data) / (clipped * (1 - clipped)) / len(y.data)

        bce.gradient_fn = gradient_fn
        bce.parents = {p}
        return bce

### 优化器（随机梯度下降）

In [21]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### 神经网络模型

In [22]:
class NNModel(Model):

    def train(self, dataset, epochs):
        self.layer.train()
        dataset.train()

        for epoch in range(epochs):
            for i in range(len(dataset)):
                features, labels = dataset[i]

                predictions = self.layer(features)
                loss = self.loss_fn(predictions, labels)
                self.optimizer.reset()
                loss.backward()
                self.optimizer.step()

    def test(self, dataset):
        self.layer.eval()
        dataset.eval()

        features, labels = dataset.items()
        predictions = self.layer(features)
        loss = self.loss_fn(predictions, labels)
        return predictions, loss

## 设置

### 学习率

In [23]:
LEARNING_RATE = 0.01

### 批大小

In [24]:
BATCH_SIZE = 2

### 卷积核大小

In [25]:
KERNEL_SIZE = 3

### 轮次

In [26]:
EPOCHS = 10

## 训练

### 迭代

让我们把卷积层应用到我们的网络模型中，作为第一层。

同时，我们需要根据卷积层的输出结果调整后续线性层的参数，来和卷积层衔接。

In [27]:
dataset = MNISTDataset('tinymnist.npz', BATCH_SIZE)
feature, label = dataset[0]
_, channels, rows, columns = feature.data.shape
conv_rows = rows - KERNEL_SIZE + 1
conv_columns = columns - KERNEL_SIZE + 1
layer = Sequential([Convolution(channels, 16, KERNEL_SIZE),
                    Flatten(),
                    Dropout(),
                    Linear(conv_rows * conv_columns * 16, 64),
                    ReLU(),
                    Linear(64, dataset.shape()[1])])
loss_fn = CELoss()
optimizer = SGDOptimizer(layer.parameters, lr=LEARNING_RATE)

model = NNModel(layer, loss_fn, optimizer)
model.train(dataset, EPOCHS)
print(layer)

Convolution[weight(16, 1, 3, 3); bias(16,); kernels=16, kernel size=3]
Flatten[]
Dropout[rate=0.2]
Linear[weight(64, 10816); bias(64,)]
ReLU[]
Linear[weight(10, 64); bias(10,)]


## 验证

### 测试

In [28]:
predictions, loss = model.test(dataset)
accuracy = dataset.estimate(predictions)
print(f'accuracy: {accuracy:.2%}')

accuracy: 94.20%


看得出，卷积层大幅提升了准确率。但是也要慢很多。我们可以观察到线性层的神经元数量已经达到几十万个。这对于我们这个小模型已经是很大的数字，可以称为“参数爆炸”了。如何解决这个问题呢？

## 课后练习

* 试试改变卷积核大小，对准确率会有什么影响？
* 能不能在卷积层后再加一个卷积层？试试看能不能做到？